In [2]:
#ipython magic
%reset -f
%matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#general imports
import sys
sys.path.append(r'C:\Users\Robert Lees\Documents\Code\Vape\suite2p_etc')
sys.path.append('..')

import numpy as np
import os
import time

In [4]:
#notebook specific imports
from utils.gsheets_importer import gsheet2df, correct_behaviour_df, split_df, df_bool, df_col, path_conversion
from utils.artifact_removal import artifact_removal
from utils.utils_funcs import *
from utils.paq2py import *
import tifffile
import copy
import math

import suite2p
print(suite2p.__path__)
from suite2p.run_s2p import run_s2p
from settings import ops

import matplotlib.pyplot as plt

['C:\\ProgramData\\Anaconda3\\lib\\site-packages\\suite2p']


In [5]:
# read g sheets to populate some lists for processing stim artifact and running suite2p

sheet_ID = '1Z9CvuA1qlLsB0Gar48bkZscLhtlP9iIWk4PYZur8cvc'
SHEET_NAME = '2019-05-23_artifact_removal_s2p'
df = gsheet2df(sheet_ID, HEADER_ROW=4, SHEET_NAME=SHEET_NAME)

# at this point we have lots of files that could be whisker stim or photostim, need to find out which is which

for_processing = split_df(df, 's2p_me') # only files with TRUE in suite2p_me column

if not for_processing.shape[0]:
    raise Exception('ERROR: no files set for processing')

stim = for_processing.loc[:,'stim'] # find out what stims have been carried out
photostim_idx = [i for i,stim in enumerate(stim) if stim=='p'] # row indices of all photostim exps (for artifact removal)
whisker_stim_idx = [i for i,stim in enumerate(stim) if stim=='w2p'] # '' for whisker stim (no artifact removal)

tiff_paths = for_processing.loc[:,'tiff_path']
paq_paths = for_processing.loc[:,'paq_path']

if not all(tiff_paths) or not all(paq_paths):
    raise Exception('ERROR: missing tiff or paq paths')
    
# below is information that will be later required for suite2p and stim removal

n_frames = [int(i) for i in list(for_processing.loc[:,'n_frames'])] # for stim removal
stim_dur = [int(i) for i in list(for_processing.loc[:,'total_stim_duration'])] # for stim removal
n_planes = [int(i) for i in list(for_processing.loc[:,'n_planes'])] # for s2p and stim removal

if not all(n_frames) or not all(stim_dur) or not all(n_planes):
    raise Exception('ERROR: missing important metadata for processing')

packerstation_path = r"P:" # the path to PackerStation on the local machine

tiffs_pstation = path_conversion(tiff_paths, packerstation_path) # convert paths (from Packer1 or PackerStation) to local PackerStation paths
paqs_pstation = path_conversion(paq_paths, packerstation_path)

tiffs_pstation

['P:rlees\\Data\\2019-05-03\\RL035\\2019-05-03_RL035_t-002\\MPTIFF',
 'P:rlees\\Data\\2019-05-09\\RL035\\2019-05-09_RL035_t-002']

In [9]:
# use first frame of tiffs to find resolution of image, required later for calculating cell diameter in pixels for s2p

frame_size = [] 

for tiff_path in tiffs_pstation:
    frame = []
    if '.tif' in tiff_path or '.tiff' in tiff_path: # the path points directly to a file
        frame = tifffile.imread(tiff_path, key=0) 
        frame_size.append(frame.shape)
        break
    else:
        for item in os.listdir(tiff_path): # the path points to a folder
            if '.tif' in item or '.tiff' in item:
                filename = os.path.join(tiff_path, item)
                frame = tifffile.imread(filename, key=0)
                frame_size.append(frame.shape)
                break
        if frame == []: 
            print(tiff_path)
            raise Exception('ERROR: could not load a TIFF file')

frame_size

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


[(512, 512), (512, 512)]

In [10]:
# obtain list of tiffs to run stim removal on

tiff_lists = []

# need to find single multi-page TIFF or many TIFFs/MPTIFFs

for tiff in tiffs_pstation:
    print(tiff)
    if '.tif' not in tiff or '.tiff' not in tiff: # if the provided path is not directly a TIFF, it may be folder with MPTIFF or multiple TIFFs/MPTIFFs
        items = os.listdir(tiff)
        newlist = []
        
        for name in items:
            if name.endswith(".tiff") or name.endswith(".tif"):
                filename = os.path.join(tiff, name)
                newlist.append(filename)
        tiff_lists.append(newlist)
        
    else:
        tiff_lists.append(tiff)

P:rlees\Data\2019-05-03\RL035\2019-05-03_RL035_t-002\MPTIFF
P:rlees\Data\2019-05-09\RL035\2019-05-09_RL035_t-002


In [14]:
# load up paq file and find stim times + start frames

remove_frames = []

for i,_ in enumerate(tiff_lists):
    paq = paq_read(paqs_pstation[i])
    stim_frames = stim_start_frame(paq, 'markpoints2packio')
    frame_clock = paq_data(paq, 'frame_clock', threshold_ttl=True)

    duration_ms = stim_dur[i] #length of stim
    duration_samples = (int(duration_ms) / 1000) * paq['rate']
    
    # use this to remove frames during stim based on paq
    to_remove = []

    for stim_frame in stim_frames: 

        # frame indexes that fall during the stim
        in_stim = np.where((frame_clock >= stim_frame) & (frame_clock <= stim_frame + duration_samples))[0]

        #empircal observation, these are the frames with artifact
        in_stim = np.append(in_stim, in_stim[-1]+1)
        in_stim = np.append(in_stim, in_stim[-1]+1)

        to_remove.append(in_stim)

    remove_frames.append(np.ravel(to_remove))

remove_frames

[array([  85,   86,   87,   88,   89,   90,   91,   92,   93,   94,  384,
         385,  386,  387,  388,  389,  390,  391,  392,  393,  682,  683,
         684,  685,  686,  687,  688,  689,  690,  691,  981,  982,  983,
         984,  985,  986,  987,  988,  989,  990, 1280, 1281, 1282, 1283,
        1284, 1285, 1286, 1287, 1288, 1289, 1579, 1580, 1581, 1582, 1583,
        1584, 1585, 1586, 1587, 1588, 1877, 1878, 1879, 1880, 1881, 1882,
        1883, 1884, 1885, 1886, 2176, 2177, 2178, 2179, 2180, 2181, 2182,
        2183, 2184, 2185, 2475, 2476, 2477, 2478, 2479, 2480, 2481, 2482,
        2483, 2484, 2774, 2775, 2776, 2777, 2778, 2779, 2780, 2781, 2782,
        2783], dtype=int64),
 array([  74,   75,   76,   77,   78,   79,   80,   81,   82,   83,   84,
          85,   86,   87,   88,   89,   90,   91,   92,   93,   94,   95,
          96,   97,   98,   99,  100,  101,  102,  103,  104,  105,  106,
         107,  108,  109,  110,  111,  112,  113,  114,  115,  116,  117,
         

In [23]:
# load up TIFFs in blocks and run stim removal on them
user_block_size = 500 # should be above 100 to work without problems

for i,tiff_list in enumerate(tiff_lists): # for each list of TIFFs

    parent_dir = os.path.dirname(tiff_list[0]) # parent directory to create output directory inside of
    output_dir = os.path.join(parent_dir, 'Artifact_removed')
    print(output_dir)
    
    try:
        os.mkdir(output_dir) # make output directory for artifact removal TIFFs
    except:
        pass
    
    base_path = os.path.splitext(tiff_list[0])[0] # get parent folder of TIFF for saving files
    base_filename = os.path.basename(base_path)
    print(base_filename)
    
    substack = np.empty((0,512,512), dtype=np.uint16) # empty np array for appending to
    
    block_size = myround(user_block_size, base=n_planes[i]) # want the block_size to be in multiples of the number of planes if multiplanar TIFF
    
    iteration = 0
    prev_frames = 0
    current_frames = 0
    
    width_thresh = 5 * (frame_size[i][0] / 512) # currently using 5 pixels as default for 512 image, scaled for other resolution
    
    if len(tiff_list)>1: # multiple TIFFs or MPTIFFs
        print('Multiple TIFF files')
        tiff_list = sorted(tiff_list) 
        
        for tiff in tiff_list:
            
            temp_tiff = tifffile.imread(tiff)
            substack = np.append(substack, temp_tiff, axis=0)
            
            if substack.shape[0]>block_size or tiff==tiff_list[-1]: # if substack is large (RAM conservation) or at last TIFF in list, process the stack
                
                substack_frames = substack.shape[0] # number of frames in current stack
                current_frames = substack_frames + prev_frames
                remove_me = [frame - prev_frames for frame in remove_frames[i] if prev_frames <= frame < current_frames] # slice the frames to be removed for this substack
                
                non_stim_frames = [non_stim for non_stim,_ in enumerate(substack) if non_stim not in remove_me]
                thresh_list = non_stim_frames[:50]
                
                if remove_me:
                    substack_processed = artifact_removal(substack, thresh_list=thresh_list, remove_me=remove_me, width_thresh=width_thresh, nplanes=n_planes[i]) # remove artifact
                else:
                    substack_processed = substack
                
                filename = base_filename + '_artifactRemoved' + str(iteration) + '.tiff'
                output_path = os.path.join(output_dir, filename)
                tifffile.imwrite(output_path, substack_processed) # write processed tiff
                
                substack = np.empty((0,512,512), dtype=np.uint16)
                iteration += 1
                prev_frames = current_frames # save which frame we are at (for calculating stims to include next time)
                    
    else: # single, large MPTIFF
        print('Single TIFF file')
        remaining_frames = n_frames[i]
        
        while remaining_frames:
            
            if remaining_frames > block_size: # load a block of TIFF pages unless there are fewer than a block left, should pre-define block according to RAM capacity
                current_frames += block_size
            else:
                current_frames += remaining_frames # load all remaining TIFF pages
            
            substack = tifffile.imread(tiff_list, key=range(prev_frames, current_frames)) # load in all frames for this block starting from the previous block
            
            remove_me = [frame - prev_frames for frame in remove_frames[i] if prev_frames <= frame < current_frames] # slice the frames to be removed so they are for this substack only
            
            non_stim_frames = [non_stim for non_stim,_ in enumerate(substack) if non_stim not in remove_me] # sometimes can't find threshold so find non-stim frames and provide it to artifact removal
            thresh_list = non_stim_frames[:50] # list of frames for threshold in artifact removal function
            
            if remove_me:
                substack_processed = artifact_removal(substack, thresh_list=thresh_list, remove_me=remove_me, width_thresh=width_thresh, nplanes=n_planes[i]) # remove artifact
            else:
                substack_processed = substack # if no frames to be removed
                    
            filename = base_filename + '_artifactRemoved' + str(iteration) + '.tiff' # construct filename
            output_path = os.path.join(output_dir, filename)
            tifffile.imwrite(output_path, substack_processed) # write processed tiff
            iteration += 1 
            prev_frames = current_frames # save progress in terms of frames with artifact removed
                        
            remaining_frames = n_frames[i] - current_frames # frames remaining to load

P:rlees\Data\2019-05-03\RL035\2019-05-03_RL035_t-002\MPTIFF\Artifact_removed
2019-05-03_RL035_t-002_Cycle00658_Ch3
Multiple TIFF files
P:rlees\Data\2019-05-09\RL035\2019-05-09_RL035_t-002\Artifact_removed
2019-05-09_RL035_t-002_Cycle00001_Ch3
Single TIFF file


In [ ]:
# sampling rate
fps = for_processing.loc[:,'fps'] 
sampling_rate = fps/n_planes
print(sampling_rate)

# cell diameter
zoom = for_processing.loc[:,'zoom'] # for s2p cell diameter calculation